In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import math
import time
import pickle
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import sys
sys.path.append('../src')
from preprocessing import *
from utils import *
from sklearn.linear_model import SGDClassifier
from sklearn.kernel_approximation import RBFSampler
from sklearn.kernel_approximation import AdditiveChi2Sampler
from sklearn.kernel_approximation import SkewedChi2Sampler

In [4]:
df_db = group_datafiles_byID('../datasets/raw/HT_Sensor_metadata.dat', '../datasets/raw/HT_Sensor_dataset.dat')
df_db = reclassify_series_samples(df_db)

In [5]:
df_train, df_test = split_series_byID(100, 0.75, df_db)

In [6]:
features = ['R1', 'R2', 'R3', 'R4', 'R5', 'R5', 'R6', 'R7', 'R8', 'Temp.', 'Humidity']
xtrain, ytrain = df_train[features].values, df_train['class'].values
xtest, ytest = df_test[features].values, df_test['class'].values
print(xtrain.shape)
print(ytrain.shape)
print(xtest.shape)
print(ytest.shape)

(707258, 11)
(707258,)
(221733, 11)
(221733,)


## Radial Basis Function

In [7]:
rbf_feature = RBFSampler(gamma=1, random_state=1)
X_features = rbf_feature.fit_transform(xtrain)
clf = SGDClassifier(max_iter=100, tol=1e-4)
clf.fit(X_features, ytrain)

X_features = rbf_feature.fit_transform(xtest)
ytrue = clf.predict(X_features)
metric_report(ytest, ytrue)

0.8198779613318721
TEST SET PROPORTIONS:
	Real background percentage: 0.8198779613318721
	Real banana percentage: 0.06752716104504065
	Real wine percentage: 0.11259487762308723
------------------------------------------
Accuracy: 0.8198779613318721
Recall on background: 1.0
Recall on banana: 0.0
Recall on wine: 0.0
F1-score: 0.738730712454762


## Additive Chi 2

In [11]:
chi2sampler = AdditiveChi2Sampler(sample_steps=2)
X_transformed = chi2sampler.fit_transform(xtrain, ytrain)
clf = SGDClassifier(max_iter=100, tol=1e-4)
clf.fit(X_transformed, ytrain)

X_features = chi2sampler.fit_transform(xtest, ytest)
ytrue = clf.predict(X_features)
metric_report(ytest, ytrue)

TEST SET PROPORTIONS:
	Real background percentage: 0.8198779613318721
	Real banana percentage: 0.06752716104504065
	Real wine percentage: 0.11259487762308723
------------------------------------------
Accuracy: 0.8621269725300249
Recall on background: 0.9890040375369924
Recall on banana: 0.0
Recall on wine: 0.4552992069214131
F1-score: 0.8230684883087167


## Skewed Chi 2

In [10]:
chi2_feature = SkewedChi2Sampler(skewedness=.01, n_components=10, random_state=0)
X_features = chi2_feature.fit_transform(xtrain, ytrain)
clf = SGDClassifier(max_iter=100, tol=1e-4)
clf.fit(X_features, ytrain)

X_features = chi2_feature.fit_transform(xtest, ytest)
ytrue = clf.predict(X_features)
metric_report(ytest, ytrue)

0.8704838702403341
TEST SET PROPORTIONS:
	Real background percentage: 0.8198779613318721
	Real banana percentage: 0.06752716104504065
	Real wine percentage: 0.11259487762308723
------------------------------------------
Accuracy: 0.8704838702403341
Recall on background: 0.9940867135328999
Recall on banana: 0.0
Recall on wine: 0.49250981334615074
F1-score: 0.831826355911274


## Hiperparámetros SKEWED

In [12]:
df_db = group_datafiles_byID('../datasets/raw/HT_Sensor_metadata.dat', '../datasets/raw/HT_Sensor_dataset.dat')
df_db = reclassify_series_samples(df_db)

In [13]:
SKEW = [.001, .005, .01, .03, .05, .07, .1, 1]
NCOMP = [5, 10, 20, 40, 80, 100]
MITER = [10, 20, 50, 70, 100, 200, 500]

In [16]:
for skew in SKEW:
    for ncomp in NCOMP:
        for miter in MITER:
            df_train, df_test = split_series_byID(100, 0.75, df_db)
            features = ['R1', 'R2', 'R3', 'R4', 'R5', 'R5', 'R6', 'R7', 'R8', 'Temp.', 'Humidity']
            xtrain, ytrain = df_train[features].values, df_train['class'].values
            xtest, ytest = df_test[features].values, df_test['class'].values
            
            chi2_feature = SkewedChi2Sampler(skewedness=skew, n_components=ncomp, random_state=0)
            clf = SGDClassifier(max_iter=miter, tol=1e-4)
            
            start_t = time.time()
            X_features = chi2_feature.fit_transform(xtrain, ytrain)
            clf.fit(X_features, ytrain)
            end_t = time.time()

            X_features = chi2_feature.fit_transform(xtest, ytest)
            ytrue = clf.predict(X_features)
            print("\n\n")
            print(skew)
            print(ncomp)
            print(miter)
            print((end_t-start_t)/60)
            metric_report(ytest, ytrue)
            

/home/man3/.local/lib/python3.8/site-packages/sklearn/linear_model/_stochastic_gradient.py:570: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn("Maximum number of iteration reached before "





0.001
5
10
0.04419084390004476
TEST SET PROPORTIONS:
	Real background percentage: 0.8051351226807536
	Real banana percentage: 0.11384991768219883
	Real wine percentage: 0.08101495963704762
------------------------------------------
Accuracy: 0.8054409343594083
Recall on background: 1.0
Recall on banana: 0.0
Recall on wine: 0.003774755675060758
F1-score: 0.7189515641226172



0.001
5
20
0.04759633541107178
TEST SET PROPORTIONS:
	Real background percentage: 0.8241633114103007
	Real banana percentage: 0.06655705150394432
	Real wine percentage: 0.10927963708575489
------------------------------------------
Accuracy: 0.8248191805279746
Recall on background: 0.9998894722298978
Recall on banana: 0.0
Recall on wine: 0.006835326970366357
F1-score: 0.7474577554323888



0.001
5
50
0.05378348429997762
TEST SET PROPORTIONS:
	Real background percentage: 0.8099535192828918
	Real banana percentage: 0.10384696994567241
	Real wine percentage: 0.08619951077143578
-------------------------------------

/home/man3/.local/lib/python3.8/site-packages/sklearn/linear_model/_stochastic_gradient.py:570: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn("Maximum number of iteration reached before "





0.001
10
10
0.0481778621673584
TEST SET PROPORTIONS:
	Real background percentage: 0.7786979227510893
	Real banana percentage: 0.08947183966978622
	Real wine percentage: 0.13183023757912454
------------------------------------------
Accuracy: 0.8377617112965824
Recall on background: 0.9942103069870816
Recall on banana: 0.0
Recall on wine: 0.4822278381305589
F1-score: 0.7886853428908692



0.001
10
20
0.05161068836847941
TEST SET PROPORTIONS:
	Real background percentage: 0.7637695245890458
	Real banana percentage: 0.14867676147602482
	Real wine percentage: 0.0875537139349294
------------------------------------------
Accuracy: 0.7795810313075506
Recall on background: 0.9668177403690514
Recall on banana: 0.013160912948732654
Recall on wine: 0.44770669003797836
F1-score: 0.7137097594454969



0.001
10
50
0.05149366060892741
TEST SET PROPORTIONS:
	Real background percentage: 0.8194279523293608
	Real banana percentage: 0.05404550379198267
	Real wine percentage: 0.12652654387865656
-------

/home/man3/.local/lib/python3.8/site-packages/sklearn/linear_model/_stochastic_gradient.py:570: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn("Maximum number of iteration reached before "


...




1
40
10
0.08369661172231038
TEST SET PROPORTIONS:
	Real background percentage: 0.7960481441783419
	Real banana percentage: 0.07194089162476011
	Real wine percentage: 0.132010964196898
------------------------------------------
Accuracy: 0.823344249515974
Recall on background: 0.983571048430016
Recall on banana: 0.0
Recall on wine: 0.3058408215661104
F1-score: 0.7728247550250086



1
40
20
0.08203249375025432
TEST SET PROPORTIONS:
	Real background percentage: 0.7590588734901877
	Real banana percentage: 0.10541531576411405
	Real wine percentage: 0.13552581074569822
------------------------------------------
Accuracy: 0.8228037795327034
Recall on background: 0.9780840488025305
Recall on banana: 0.0
Recall on wine: 0.5931003314251281
F1-score: 0.7718246940858284



1
40
50
0.09117622375488281
TEST SET PROPORTIONS:
	Real background percentage: 0.7797572055632497
	Real banana percentage: 0.10141246633494083
	Real wine percentage: 0.11883032810180945
---------------------------------------

/home/man3/.local/lib/python3.8/site-packages/sklearn/linear_model/_stochastic_gradient.py:570: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn("Maximum number of iteration reached before "





1
80
10
0.1255902926127116
TEST SET PROPORTIONS:
	Real background percentage: 0.8199172183268701
	Real banana percentage: 0.07345894211811405
	Real wine percentage: 0.10662383955501586
------------------------------------------
Accuracy: 0.8528383842340519
Recall on background: 0.9962788969397776
Recall on banana: 0.0
Recall on wine: 0.33737447954935096
F1-score: 0.8048328421234853



1
80
20
0.1298347036043803
TEST SET PROPORTIONS:
	Real background percentage: 0.8256339898207568
	Real banana percentage: 0.09957955299845099
	Real wine percentage: 0.07478645718079222
------------------------------------------
Accuracy: 0.8610666076565612
Recall on background: 0.9968855868605
Recall on banana: 0.0
Recall on wine: 0.5081666469404663
F1-score: 0.81065341987466



1
80
50
0.12625283400217693
TEST SET PROPORTIONS:
	Real background percentage: 0.7710886806056236
	Real banana percentage: 0.08477067273029781
	Real wine percentage: 0.14414064666407853
-----------------------------------------

/home/man3/.local/lib/python3.8/site-packages/sklearn/linear_model/_stochastic_gradient.py:570: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn("Maximum number of iteration reached before "





1
100
10
0.14337761004765828
TEST SET PROPORTIONS:
	Real background percentage: 0.7874423657851503
	Real banana percentage: 0.05776229088275302
	Real wine percentage: 0.15479534333209674
------------------------------------------
Accuracy: 0.8314651898175137
Recall on background: 0.9814805467220792
Recall on banana: 0.0
Recall on wine: 0.37860199714693293
F1-score: 0.7896095742628172



1
100
20
0.15319114923477173
TEST SET PROPORTIONS:
	Real background percentage: 0.8018832294724554
	Real banana percentage: 0.0886079208940619
	Real wine percentage: 0.10950884963348269
------------------------------------------
Accuracy: 0.8351358774956653
Recall on background: 0.9744650560366512
Recall on banana: 0.0
Recall on wine: 0.49063332810785387
F1-score: 0.7894170331968606



1
100
50
0.15635459423065184
TEST SET PROPORTIONS:
	Real background percentage: 0.8254634472587588
	Real banana percentage: 0.05772297772366368
	Real wine percentage: 0.11681357501757755
-------------------------------

## Hiperparámetros ADDITIVE

In [17]:
df_db = group_datafiles_byID('../datasets/raw/HT_Sensor_metadata.dat', '../datasets/raw/HT_Sensor_dataset.dat')
df_db = reclassify_series_samples(df_db)

In [23]:
SSTEP = [1, 2, 3]
MITER = [100, 200, 500]

In [24]:
for sstep in SSTEP:
    for miter in MITER:
        df_train, df_test = split_series_byID(100, 0.75, df_db)
        features = ['R1', 'R2', 'R3', 'R4', 'R5', 'R5', 'R6', 'R7', 'R8', 'Temp.', 'Humidity']
        xtrain, ytrain = df_train[features].values, df_train['class'].values
        xtest, ytest = df_test[features].values, df_test['class'].values
        
        chi2sampler = AdditiveChi2Sampler(sample_steps=sstep)
        clf = SGDClassifier(max_iter=miter, tol=1e-4)
        
        start_t = time.time()
        X_features = chi2sampler.fit_transform(xtrain, ytrain)
        clf.fit(X_features, ytrain)
        end_t = time.time()
        
        X_features = chi2sampler.fit_transform(xtest, ytest)
        ytrue = clf.predict(X_features)
        print("\n\n")
        print(sstep)
        print(miter)
        print((end_t-start_t)/60)
        metric_report(ytest, ytrue)




1
100
0.35735353231430056
TEST SET PROPORTIONS:
	Real background percentage: 0.7955920630091606
	Real banana percentage: 0.07593888381840923
	Real wine percentage: 0.12846905317243013
------------------------------------------
Accuracy: 0.8017695974039144
Recall on background: 0.9686012916383413
Recall on banana: 0.0
Recall on wine: 0.2425338771214314
F1-score: 0.7497379526455471



1
200
0.38767033418019614
TEST SET PROPORTIONS:
	Real background percentage: 0.781992573152661
	Real banana percentage: 0.07377751593380108
	Real wine percentage: 0.14422991091353793
------------------------------------------
Accuracy: 0.8203184974916538
Recall on background: 0.9725695891145901
Recall on banana: 0.0
Recall on wine: 0.4144514931146526
F1-score: 0.7761206250258254



1
500
0.28893452882766724
TEST SET PROPORTIONS:
	Real background percentage: 0.7513023953984131
	Real banana percentage: 0.13457034617017966
	Real wine percentage: 0.11412725843140717
------------------------------------------

## Hiperparámetros RBF

In [17]:
df_db = group_datafiles_byID('../datasets/raw/HT_Sensor_metadata.dat', '../datasets/raw/HT_Sensor_dataset.dat')
df_db = reclassify_series_samples(df_db)

In [25]:
GM = [1, 2, 5, 7, 10, 15, 20, 40, 70, 100]
MITER = [100, 200, 500]

In [26]:
for gm in GM:
    for miter in MITER:
        df_train, df_test = split_series_byID(100, 0.75, df_db)
        features = ['R1', 'R2', 'R3', 'R4', 'R5', 'R5', 'R6', 'R7', 'R8', 'Temp.', 'Humidity']
        xtrain, ytrain = df_train[features].values, df_train['class'].values
        xtest, ytest = df_test[features].values, df_test['class'].values
        
        rbf_feature = RBFSampler(gamma=gm, random_state=1)
        clf = SGDClassifier(max_iter=miter, tol=1e-4)
        
        start_t = time.time()
        X_features = rbf_feature.fit_transform(xtrain)
        clf.fit(X_features, ytrain)
        end_t = time.time()
        
        X_features = rbf_feature.fit_transform(xtest)
        ytrue = clf.predict(X_features)
        print("\n\n")
        print(sstep)
        print(miter)
        print((end_t-start_t)/60)
        metric_report(ytest, ytrue)




3
100
0.17455610036849975
TEST SET PROPORTIONS:
	Real background percentage: 0.7983156646673176
	Real banana percentage: 0.05801018273120379
	Real wine percentage: 0.1436741526014786
------------------------------------------
Accuracy: 0.7983156646673176
Recall on background: 1.0
Recall on banana: 0.0
Recall on wine: 0.0
F1-score: 0.7087831274284327



3
200
0.16665894587834676
TEST SET PROPORTIONS:
	Real background percentage: 0.7932434257165041
	Real banana percentage: 0.05734986328190933
	Real wine percentage: 0.14940671100158662
------------------------------------------
Accuracy: 0.7932434257165041
Recall on background: 1.0
Recall on banana: 0.0
Recall on wine: 0.0
F1-score: 0.7017844018483315



3
500
0.16541130940119425
TEST SET PROPORTIONS:
	Real background percentage: 0.8002565520291939
	Real banana percentage: 0.08917837000995245
	Real wine percentage: 0.1105650779608537
------------------------------------------
Accuracy: 0.8002565520291939
Recall on background: 1.0
Recal

F1-score: 0.6991219025111794



3
200
0.16857704321543376
TEST SET PROPORTIONS:
	Real background percentage: 0.8138721176486063
	Real banana percentage: 0.09243693794421863
	Real wine percentage: 0.09369094440717503
------------------------------------------
Accuracy: 0.8138721176486063
Recall on background: 1.0
Recall on banana: 0.0
Recall on wine: 0.0
F1-score: 0.7303577991424294



3
500
0.17750498056411743
TEST SET PROPORTIONS:
	Real background percentage: 0.8102909166292368
	Real banana percentage: 0.052862522889817916
	Real wine percentage: 0.13684656048094532
------------------------------------------
Accuracy: 0.8102909166292368
Recall on background: 1.0
Recall on banana: 0.0
Recall on wine: 0.0
F1-score: 0.7253766381310528



3
100
0.1759070078531901
TEST SET PROPORTIONS:
	Real background percentage: 0.8301316902259273
	Real banana percentage: 0.05379028902044487
	Real wine percentage: 0.11607802075362786
------------------------------------------
Accuracy: 0.8301316902259273


In [28]:
df_db = group_datafiles_byID('../datasets/raw/HT_Sensor_metadata.dat', '../datasets/raw/HT_Sensor_dataset.dat')
df_db = reclassify_series_samples(df_db)
df_db = remove_excess_bg(df_db)

In [29]:
SKEW = [.01, .03, .05, .07, .1]
NCOMP = [40, 80, 100, 200]
MITER = [100, 200, 500]

In [30]:
for skew in SKEW:
    for ncomp in NCOMP:
        for miter in MITER:
            df_train, df_test = split_series_byID(100, 0.75, df_db)
            features = ['R1', 'R2', 'R3', 'R4', 'R5', 'R5', 'R6', 'R7', 'R8', 'Temp.', 'Humidity']
            xtrain, ytrain = df_train[features].values, df_train['class'].values
            xtest, ytest = df_test[features].values, df_test['class'].values
            
            chi2_feature = SkewedChi2Sampler(skewedness=skew, n_components=ncomp, random_state=0)
            clf = SGDClassifier(max_iter=miter, tol=1e-4)
            
            start_t = time.time()
            X_features = chi2_feature.fit_transform(xtrain, ytrain)
            clf.fit(X_features, ytrain)
            end_t = time.time()

            X_features = chi2_feature.fit_transform(xtest, ytest)
            ytrue = clf.predict(X_features)
            print("\n\n")
            print(skew)
            print(ncomp)
            print(miter)
            print((end_t-start_t)/60)
            metric_report(ytest, ytrue)
            




0.01
40
100
0.1350493311882019
TEST SET PROPORTIONS:
	Real background percentage: 0.6109081591085297
	Real banana percentage: 0.21817632294773914
	Real wine percentage: 0.1709155179437312
------------------------------------------
Accuracy: 0.6137728485262618
Recall on background: 0.901311048731434
Recall on banana: 0.0
Recall on wine: 0.36950755391535
F1-score: 0.5265656577965279



0.01
40
200
0.13335659901301067
TEST SET PROPORTIONS:
	Real background percentage: 0.7097024940538673
	Real banana percentage: 0.1448597509932367
	Real wine percentage: 0.145437754952896
------------------------------------------
Accuracy: 0.7338723607807704
Recall on background: 0.9486533798900986
Recall on banana: 0.0
Recall on wine: 0.4167466081951487
F1-score: 0.6659669041254962



0.01
40
500
0.1317954619725545
TEST SET PROPORTIONS:
	Real background percentage: 0.7603343220772389
	Real banana percentage: 0.1376083112935985
	Real wine percentage: 0.1020573666291626
----------------------------------




0.03
80
500
0.20221633116404217
TEST SET PROPORTIONS:
	Real background percentage: 0.7044958188453692
	Real banana percentage: 0.10755337797063134
	Real wine percentage: 0.18795080318399954
------------------------------------------
Accuracy: 0.7576941866145579
Recall on background: 0.9477988639955947
Recall on banana: 0.007881905016364972
Recall on wine: 0.4741992202431007
F1-score: 0.7051248647993051



0.03
100
100
0.23272937933603924
TEST SET PROPORTIONS:
	Real background percentage: 0.6579465223576966
	Real banana percentage: 0.19913840098924332
	Real wine percentage: 0.14291507665306014
------------------------------------------
Accuracy: 0.6972303846614102
Recall on background: 0.953833095880445
Recall on banana: 0.0
Recall on wine: 0.487416848862632
F1-score: 0.6120337410202655



0.03
100
200
0.2527662873268127
TEST SET PROPORTIONS:
	Real background percentage: 0.7146009712639372
	Real banana percentage: 0.17192597688310096
	Real wine percentage: 0.1134730518529618
--------

F1-score: 0.7540350349194233



0.05
200
200
0.37310070196787515
TEST SET PROPORTIONS:
	Real background percentage: 0.6786232522816963
	Real banana percentage: 0.15427385779573902
	Real wine percentage: 0.16710288992256472
------------------------------------------
Accuracy: 0.7324956942730441
Recall on background: 0.9270398273415799
Recall on banana: 0.0027250351617440225
Recall on wine: 0.6161743223502678
F1-score: 0.6703499853527853



0.05
200
500
0.3774741530418396
TEST SET PROPORTIONS:
	Real background percentage: 0.5820362182787233
	Real banana percentage: 0.19003492621080895
	Real wine percentage: 0.22792885551046776
------------------------------------------
Accuracy: 0.6994865375944657
Recall on background: 0.9335992600300613
Recall on banana: 0.0
Recall on wine: 0.6848538529672277
F1-score: 0.6254039005396609



0.07
40
100
0.13533418973286945
TEST SET PROPORTIONS:
	Real background percentage: 0.7446138287768113
	Real banana percentage: 0.11142695101368665
	Real wine percent

F1-score: 0.6299029246056391



0.1
80
100
0.19626532395680746
TEST SET PROPORTIONS:
	Real background percentage: 0.6897107084375022
	Real banana percentage: 0.14248940342459754
	Real wine percentage: 0.16779988813790037
------------------------------------------
Accuracy: 0.7502240611333113
Recall on background: 0.9719299274066692
Recall on banana: 0.0
Recall on wine: 0.47600497971968997
F1-score: 0.6817614111854428



0.1
80
200
0.18946096102396648
TEST SET PROPORTIONS:
	Real background percentage: 0.6986522945516386
	Real banana percentage: 0.1370337693556843
	Real wine percentage: 0.16431393609267714
------------------------------------------
Accuracy: 0.7539031033027734
Recall on background: 0.9680816991370712
Recall on banana: 0.0
Recall on wine: 0.4719660713590911
F1-score: 0.6878657353774232



0.1
80
500
0.18772615591684977
TEST SET PROPORTIONS:
	Real background percentage: 0.64573342316991
	Real banana percentage: 0.12657709431519426
	Real wine percentage: 0.2276894825148957


In [31]:
df_db = group_datafiles_byID('../datasets/raw/HT_Sensor_metadata.dat', '../datasets/raw/HT_Sensor_dataset.dat')
df_db = reclassify_series_samples(df_db)
df_db.loc[df_db['class']!='background','class'] = 'not-background'
df_db[df_db['class']!='background'].head()

,time,R1,R2,R3,R4,R5,R6,R7,R8,Temp.,Humidity,id,date,class,t0,dt
3429,13.490021,12.8102,10.3665,10.4529,11.6742,13.4941,13.2749,8.30531,9.04553,26.4234,59.4725,0,07-04-15,not-background,13.49,1.64
3430,13.490309,12.8097,10.3655,10.4523,11.6734,13.4934,13.2740,8.30527,9.04545,26.4241,59.4745,0,07-04-15,not-background,13.49,1.64
3431,13.490587,12.8088,10.3645,10.4516,11.6731,13.4930,13.2730,8.30523,9.04538,26.4246,59.4763,0,07-04-15,not-background,13.49,1.64
3432,13.490865,12.8080,10.3638,10.4508,11.6727,13.4922,13.2719,8.30520,9.04516,26.4251,59.4779,0,07-04-15,not-background,13.49,1.64
3433,13.491144,12.8078,10.3628,10.4503,11.6722,13.4914,13.2708,8.30517,9.04511,26.4256,59.4793,0,07-04-15,not-background,13.49,1.64


In [32]:
SKEW = [.01, .03, .05, .07, .1]
NCOMP = [200]
MITER = [500]

In [33]:
for skew in SKEW:
    for ncomp in NCOMP:
        for miter in MITER:
            df_train, df_test = split_series_byID(100, 0.75, df_db)
            features = ['R1', 'R2', 'R3', 'R4', 'R5', 'R5', 'R6', 'R7', 'R8', 'Temp.', 'Humidity']
            xtrain, ytrain = df_train[features].values, df_train['class'].values
            xtest, ytest = df_test[features].values, df_test['class'].values
            
            chi2_feature = SkewedChi2Sampler(skewedness=skew, n_components=ncomp, random_state=0)
            clf = SGDClassifier(max_iter=miter, tol=1e-4)
            
            start_t = time.time()
            X_features = chi2_feature.fit_transform(xtrain, ytrain)
            clf.fit(X_features, ytrain)
            end_t = time.time()

            X_features = chi2_feature.fit_transform(xtest, ytest)
            ytrue = clf.predict(X_features)
            print("\n\n")
            print(skew)
            print(ncomp)
            print(miter)
            print((end_t-start_t)/60)
            metric_report(ytest, ytrue)
            




0.01
200
500
0.2824456731478373


../src/utils.py:64: RuntimeWarning: invalid value encountered in long_scalars
  recall_ban = conf_m[1][1]/sum(conf_m[1,:])   # predicted_banana / total_real_banana
../src/utils.py:65: RuntimeWarning: invalid value encountered in long_scalars
  recall_wine = conf_m[2][2]/sum(conf_m[2,:]) # predicted_wine / total_real_wine


TEST SET PROPORTIONS:
	Real background percentage: 0.8070825513905157
	Real banana percentage: 0.0
	Real wine percentage: 0.0
------------------------------------------
Accuracy: 0.8400656614256541
Recall on background: 1.0
Recall on banana: nan
Recall on wine: nan
F1-score: 0.8011473487909783



0.03
200
500
0.2877656579017639


../src/utils.py:64: RuntimeWarning: invalid value encountered in long_scalars
  recall_ban = conf_m[1][1]/sum(conf_m[1,:])   # predicted_banana / total_real_banana
../src/utils.py:65: RuntimeWarning: invalid value encountered in long_scalars
  recall_wine = conf_m[2][2]/sum(conf_m[2,:]) # predicted_wine / total_real_wine


TEST SET PROPORTIONS:
	Real background percentage: 0.7883653080306126
	Real banana percentage: 0.0
	Real wine percentage: 0.0
------------------------------------------
Accuracy: 0.8693986563701216
Recall on background: 1.0
Recall on banana: nan
Recall on wine: nan
F1-score: 0.8564566700443131



0.05
200
500
0.28721572160720826


...

TEST SET PROPORTIONS:
	Real background percentage: 0.7667710437565415
	Real banana percentage: 0.0
	Real wine percentage: 0.0
------------------------------------------
Accuracy: 0.7942766323098062
Recall on background: 1.0
Recall on banana: nan
Recall on wine: nan
F1-score: 0.7353606058868417



0.07
200
500
0.28032708962758385


../src/utils.py:64: RuntimeWarning: invalid value encountered in long_scalars
  recall_ban = conf_m[1][1]/sum(conf_m[1,:])   # predicted_banana / total_real_banana
../src/utils.py:65: RuntimeWarning: invalid value encountered in long_scalars
  recall_wine = conf_m[2][2]/sum(conf_m[2,:]) # predicted_wine / total_real_wine


TEST SET PROPORTIONS:
	Real background percentage: 0.7674202088970353
	Real banana percentage: 0.0
	Real wine percentage: 0.0
------------------------------------------
Accuracy: 0.7925119598332482
Recall on background: 1.0
Recall on banana: nan
Recall on wine: nan
F1-score: 0.7489573309027695



0.1
200
500
0.27463539838790896


../src/utils.py:64: RuntimeWarning: invalid value encountered in long_scalars
  recall_ban = conf_m[1][1]/sum(conf_m[1,:])   # predicted_banana / total_real_banana
../src/utils.py:65: RuntimeWarning: invalid value encountered in long_scalars
  recall_wine = conf_m[2][2]/sum(conf_m[2,:]) # predicted_wine / total_real_wine


TEST SET PROPORTIONS:
	Real background percentage: 0.7579896081350213
	Real banana percentage: 0.0
	Real wine percentage: 0.0
------------------------------------------
Accuracy: 0.8024774984890829
Recall on background: 1.0
Recall on banana: nan
Recall on wine: nan
F1-score: 0.7690185944708878


In [34]:
df_db = group_datafiles_byID('../datasets/raw/HT_Sensor_metadata.dat', '../datasets/raw/HT_Sensor_dataset.dat')
df_db = reclassify_series_samples(df_db)
df_db = df_db[df_db['class']!='background']

In [35]:
SKEW = [.01, .03, .05, .07, .1]
NCOMP = [200]
MITER = [500]

In [36]:
for skew in SKEW:
    for ncomp in NCOMP:
        for miter in MITER:
            df_train, df_test = split_series_byID(100, 0.75, df_db)
            features = ['R1', 'R2', 'R3', 'R4', 'R5', 'R5', 'R6', 'R7', 'R8', 'Temp.', 'Humidity']
            xtrain, ytrain = df_train[features].values, df_train['class'].values
            xtest, ytest = df_test[features].values, df_test['class'].values
            
            chi2_feature = SkewedChi2Sampler(skewedness=skew, n_components=ncomp, random_state=0)
            clf = SGDClassifier(max_iter=miter, tol=1e-4)
            
            start_t = time.time()
            X_features = chi2_feature.fit_transform(xtrain, ytrain)
            clf.fit(X_features, ytrain)
            end_t = time.time()

            X_features = chi2_feature.fit_transform(xtest, ytest)
            ytrue = clf.predict(X_features)
            print("\n\n")
            print(skew)
            print(ncomp)
            print(miter)
            print((end_t-start_t)/60)
            metric_report(ytest, ytrue)
            




0.01
200
500
0.10739375750223795


../src/utils.py:63: RuntimeWarning: invalid value encountered in long_scalars
  recall_bg = conf_m[0][0]/sum(conf_m[0,:])   # predicted_bg / total_real_bg


TEST SET PROPORTIONS:
	Real background percentage: 0.0
	Real banana percentage: 0.4506017206872352
	Real wine percentage: 0.5493982793127647
------------------------------------------
Accuracy: 0.6882748940815637
Recall on background: nan
Recall on banana: 0.7017189663128749
Recall on wine: 0.6772484269290817
F1-score: 0.6890247945257347



0.03
200
500
0.08208655913670858
TEST SET PROPORTIONS:
	Real background percentage: 0.0
	Real banana percentage: 0.6218046064287522
	Real wine percentage: 0.37819539357124776
------------------------------------------


../src/utils.py:63: RuntimeWarning: invalid value encountered in long_scalars
  recall_bg = conf_m[0][0]/sum(conf_m[0,:])   # predicted_bg / total_real_bg


Accuracy: 0.6200012655024044
Recall on background: nan
Recall on banana: 0.5687900681795054
Recall on wine: 0.7041994311527522
F1-score: 0.6252278747491659



0.05
200
500
0.09572492043177287


...

TEST SET PROPORTIONS:
	Real background percentage: 0.0
	Real banana percentage: 0.32352321281446994
	Real wine percentage: 0.6764767871855301
------------------------------------------
Accuracy: 0.8564695062367075
Recall on background: nan
Recall on banana: 0.8807817589576548
Recall on wine: 0.8448422364470628
F1-score: 0.8594444920711031



0.07
200
500
0.08560238679250082


../src/utils.py:63: RuntimeWarning: invalid value encountered in long_scalars
  recall_bg = conf_m[0][0]/sum(conf_m[0,:])   # predicted_bg / total_real_bg


TEST SET PROPORTIONS:
	Real background percentage: 0.0
	Real banana percentage: 0.5632419038838308
	Real wine percentage: 0.43675809611616917
------------------------------------------
Accuracy: 0.7840843609542468
Recall on background: nan
Recall on banana: 0.7996828482275308
Recall on wine: 0.7639685995118412
F1-score: 0.7843332300565009



0.1
200
500
0.0845176617304484


../src/utils.py:63: RuntimeWarning: invalid value encountered in long_scalars
  recall_bg = conf_m[0][0]/sum(conf_m[0,:])   # predicted_bg / total_real_bg


TEST SET PROPORTIONS:
	Real background percentage: 0.0
	Real banana percentage: 0.34552439191705375
	Real wine percentage: 0.6544756080829462
------------------------------------------
Accuracy: 0.7742890182473884
Recall on background: nan
Recall on banana: 0.5487548925066652
Recall on wine: 0.893357690464782
F1-score: 0.7651890338627108


In [37]:
df_db = group_datafiles_byID('../datasets/preprocessed/HT_Sensor_preprocessed_dataset/HT_Sensor_prep_metadata.dat', '../datasets/preprocessed/HT_Sensor_preprocessed_dataset/HT_Sensor_prep_dataset.dat')
df_db = reclassify_series_samples(df_db)
df_db = remove_excess_bg(df_db)

In [38]:
SKEW = [.01, .03, .05, .07, .1]
NCOMP = [200]
MITER = [500]

In [39]:
for skew in SKEW:
    for ncomp in NCOMP:
        for miter in MITER:
            df_train, df_test = split_series_byID(100, 0.75, df_db)
            features = ['R1', 'R2', 'R3', 'R4', 'R5', 'R5', 'R6', 'R7', 'R8', 'Temp.', 'Humidity']
            xtrain, ytrain = df_train[features].values, df_train['class'].values
            xtest, ytest = df_test[features].values, df_test['class'].values
            
            chi2_feature = SkewedChi2Sampler(skewedness=skew, n_components=ncomp, random_state=0)
            clf = SGDClassifier(max_iter=miter, tol=1e-4)
            
            start_t = time.time()
            X_features = chi2_feature.fit_transform(xtrain, ytrain)
            clf.fit(X_features, ytrain)
            end_t = time.time()

            X_features = chi2_feature.fit_transform(xtest, ytest)
            ytrue = clf.predict(X_features)
            print("\n\n")
            print(skew)
            print(ncomp)
            print(miter)
            print((end_t-start_t)/60)
            metric_report(ytest, ytrue)
            




0.01
200
500
0.37181324561436974
TEST SET PROPORTIONS:
	Real background percentage: 0.6412668963689372
	Real banana percentage: 0.16421945401537238
	Real wine percentage: 0.19451364961569043
------------------------------------------
Accuracy: 0.7441359660747416
Recall on background: 0.9329303575118826
Recall on banana: 0.10438185926404131
Recall on wine: 0.661840850252078
F1-score: 0.6976307506181539



0.03
200
500
0.3794602314631144
TEST SET PROPORTIONS:
	Real background percentage: 0.6597915628689794
	Real banana percentage: 0.11743869395778021
	Real wine percentage: 0.22276974317324033
------------------------------------------
Accuracy: 0.7645766078119561
Recall on background: 0.9683776067527309
Recall on banana: 0.0
Recall on wine: 0.5640318627450981
F1-score: 0.7087875776482109



0.05
200
500
0.3780316591262817
TEST SET PROPORTIONS:
	Real background percentage: 0.5783263410248727
	Real banana percentage: 0.12661072819898112
	Real wine percentage: 0.29506293077614626
-------